In [ ]:
## This script scrapes html from a url and extracts links
## for 'link per line', it checks cat and rep for all source links and links in html using Data Aggregator
## All tables are merged.

In [9]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
from urllib.parse import urlparse
import os
import pandas as pd
import sys
import paramiko
from scp import SCPClient, SCPException
import csv

In [10]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [11]:
headers={'User-Agent': 'Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36'}

In [13]:
# file_date = datetime.now().strftime("%Y%m%d")
in_filename = 'test.txt'
out_filename = 'test.csv'

In [73]:
## one link per line

def links_in_html_source_to_df(line):
    tag_list = ['iframe', 'embed', 'input', 'script', 'source']
    tmp_dict_list = []
    seen_urls = []
    
    url = line.strip().lower()
    
    if url.startswith("http") == False:
        domain = url
        url = "http://" + url
    else:
        domain = url.split('/')[-1]

    try:
        response = requests.get(url, headers=headers, verify=False, timeout=1)
        response_head = requests.head(url, headers=headers, verify=False, timeout=1)

        if (response.status_code >= 200 and response.status_code < 400) or (response_head.status_code >= 200 and response_head.status_code < 400):

            soup = BeautifulSoup(response.text, "lxml")

            if soup.find('title') is not None:
                title = soup.find('title').text

#                 if soup.find('a') is None and soup.find('iframe') is None and soup.find('embed') is None and soup.find('input') is None and soup.find('script') is None and soup.find('source') is None:
#                     tmp_dict = {'parent_url_title' : title,
#                                 'parent_url_content_length' : size}
#                     df = df.append(tmp_dict, ignore_index = True)   

#                 else:
#                 print(url)
#             if soup.find('href') is not None:
            for href in soup.find_all('a'):
                link=href.get('href')

                if (link is not None) and (link.endswith(('.jpg','.png'))==False) and \
                    (link.startswith(('tel','mailto','#'))==False) and (link not in seen_urls):

                    tmp_dict = {}
                    tmp_dict = {'parent_url' : url,
                                'http_response_code' : response.status_code,
                                'parent_url_title' : title,
                                'parent_url_content_length' : response.headers['content-length']}

                    if (link.startswith('http')) and (urlparse(url).netloc not in link):
                        tmp_dict['link'] = link
                        tmp_dict['link_type'] = 'external'

                    else:
                        tmp_dict['link'] = 'http://' + urlparse(url).netloc + '/' + link
                        tmp_dict['link_type'] = 'internal'
                    seen_urls.append(link)
                    tmp_dict_list.append(tmp_dict)

            for tag in tag_list:
                for link in soup.find_all(tag):

                    link = link.get('src').replace('url=','')
                    if link.startswith('//'):
                        link.replace('//','')
                    if (link is not None) and (link not in seen_urls):
                        tmp_dict = {'parent_url' : url,
                                    'http_response_code' : response.status_code,
                                    'parent_url_title' : title,
                                    'link' : link,
                                    'link_type' : tag,
                                    'parent_url_title' : title,
                                    'parent_url_content_length' : response.headers['content-length']}
                    seen_urls.append(link)
                    tmp_dict_list.append(tmp_dict)
            
#             if response.headers is not None:
#                 tmp_dict['content-length'] = response.headers['content-length']                    

        else:
            tmp_dict = {'parent_url' : url,
                        'http_response_code' : response.status_code}
            tmp_dict_list.append(tmp_dict)
            
    except:
        tmp_dict = {'parent_url' : url}
        tmp_dict_list.append(tmp_dict)

    return tmp_dict_list

In [79]:
columns = ['parent_url', 'http_response_code', 'parent_url_title', 'link', 'link_type', 'parent_url_content_length']

df_urls = pd.DataFrame(columns=columns)

with open(out_filename, 'w', newline='', encoding='utf-8-sig') as outfile:
    w = csv.writer(outfile)
    w.writerow(columns)
    
    with open(in_filename, 'r') as infile:
        for line in infile:
            dict_list = links_in_html_source_to_df(line)
            for d in dict_list:
                df_urls = df.append(d, ignore_index=True)
                w.writerow([d.get(col, None) for col in columns])

C:\Users\kdeside1\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kdeside1\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [80]:
df_urls

,parent_url,http_response_code,parent_url_title,link,link_type,parent_url_content_length
0,http://2019coronavirusnumbers.com,NaN,NaN,NaN,NaN,NaN
1,http://2ndcoronavirusoutbreak.com,NaN,NaN,NaN,NaN,NaN
2,http://2ndcoronawave.com,200,,https://www.enable-javascript.com/,external,800
3,http://2ndcoronawave.com,200,,https://www.google.com/adsense/domains/caf.js,script,800
4,http://2ndcoronawave.com,200,,https://d1hi41nc56pmug.cloudfront.net/static/js/main.314a8987.js,script,800
5,http://aftercorona-life.net,NaN,NaN,NaN,NaN,NaN
6,http://aftercoronabody.com,403,NaN,NaN,NaN,NaN
7,http://americacoronavirus19.info,NaN,NaN,NaN,NaN,NaN
8,http://americafightscoronavirus19.info,NaN,NaN,NaN,NaN,NaN
9,http://anticoronastuff.com,NaN,NaN,NaN,NaN,NaN


In [81]:
## functions to connect to Unix
def connect(host, user, pwd, remote_path):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(host, username=user, password=pwd)
    return client

def upload_single_file(remote_path, file):
    """Upload a single file to a remote directory."""
    try:
        scp.put(file,
                     recursive=True,
                     remote_path=remote_path)
        print('success')
    except SCPException as error:
        logger.error(error)
        raise error
    finally:
        return file
    
def download_file(file):
    try:
        scp.get(file)
    except SCPException as error:
        logger.error(error)
        raise error
    finally:
        return file   

In [86]:
filename = in_filename + '_url.txt'
df_urls['parent_url'].unique().tofile(filename, "\n", '%s')

In [108]:
merged_file = in_filename + '.csv'
df_merged.to_csv(merged_file, encoding='utf-8-sig')

In [109]:
filename = 'links.txt'

In [110]:
df['link'].dropna().unique().tofile(filename, "\n", '%s')

In [37]:
df_merged2.drop(['url_x', 'url_y'], axis=1, inplace=True)